In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from src import const
from pathlib import Path

Path.cwd()

2024-06-15 21:31:15.169 | INFO     | src.const:get_project_root:20 - PROJECT_ROOT set to /Users/adamzh0u/170_BI/opt_price


PosixPath('/Users/adamzh0u/170_BI/opt_price')

In [2]:
def load_data(file_path= const.PATH_DATA/'tpyrced_1159系列号利润测算模板0529 dgchen.xls'):
    df = pd.read_excel(file_path, skiprows=104, nrows=18, sheet_name='1159盈利模型-拆分deal')
    return df

In [5]:
"""
    'FBA佣金（本地国)' = '售价（本地国)' * 0.15
    '人工6%' = '售价（本地国)' * 0.06
    '仓储成本2%' = '售价（本地国)' * 0.02
"""

"\n    'FBA佣金（本地国)' = '售价（本地国)' * 0.15\n    '人工6%' = '售价（本地国)' * 0.06\n    '仓储成本2%' = '售价（本地国)' * 0.02\n"

In [4]:
df = load_data()
df.columns

Index(['ASIN', '型号', '采购成本', '运费及税金￥', '售价（本地国)', '最新亚马逊配送费（美金）', 'FBA佣金（本地国)',
       'VAT20%\n（本地国)', '折损（退货率）', '促销折扣费用', 'PPC推广费用', '人工6%', '仓储成本2%',
       '实得收入（本地国)', '汇率', '得到人民币￥', '毛利润（RMB)', '利润率', '非deal 20天日均销量',
       'Unnamed: 19', '非deal 20天日均毛利润', '非deal 20天日均销售额', 'Unnamed: 22',
       '陪卖/主卖比值', '1.20~2.10', '配送时间邮编90001', '备注', 'Unnamed: 27'],
      dtype='object')

In [6]:
df.to_json('data.json')

In [7]:
params = ['型号', '采购成本', '运费及税金￥', '最新亚马逊配送费（美金）','VAT20%\n（本地国)','折损（退货率）','促销折扣费用','PPC推广费用','汇率']
df_params = df[params]
df_params

,型号,采购成本,运费及税金￥,最新亚马逊配送费（美金）,VAT20%\n（本地国),折损（退货率）,促销折扣费用,PPC推广费用,汇率
0,115900BLK,65,10.00,7.09,0,0.1049,0,2,7.1
1,115900BLU,65,10.00,7.09,0,0.1213,0,0,7.1
2,115900PUR,67,10.00,7.09,0,0.1218,0,0,7.1
3,115900BBU,65,10.00,7.09,0,0.0940,0,0,7.1
4,115900PNK,63,10.00,7.17,0,0.0817,0,0,7.1
5,115900WHT,63,10.00,7.17,0,0.1239,0,0,7.1
6,115900PBU,69,10.00,7.17,0,0.1161,0,0,7.1
7,115900AGN,69,10.00,7.17,0,0.0815,0,0,7.1
8,115900LPR,69,10.00,7.17,0,0.0765,0,0,7.1
9,115902BLK,65,10.00,6.77,0,0.1513,0,0,7.1


In [15]:
p_lbound = 34
p_ubound = [35,40,45,41,42,43,44,45,46,47,48,49,44,43,42,43,44,54]
n_product = 18
l_lsale =[127,24,7,4,4,2,7,11,4,18,8,4,12,9,3,4,3,2] 
l_usale = [60,20,4,2,2,1,4,1,2,10,4,2,6,4,1,2,1,1]

df_variables = pd.DataFrame(columns=['型号', '价格下界', '价格上界', '价格下界时销量', '价格上界时销量'])
df_variables['型号'] = df_params['型号']
df_variables['价格下界'] = p_lbound
df_variables['价格上界'] = p_ubound
df_variables['价格下界时销量'] = l_lsale
df_variables['价格上界时销量'] = l_usale
df_variables.iloc[0,2] = 35

# df_variables['slope'] =  (df_variables['l_usale'] - df_variables['l_lsale']) / (df_variables['p_ubound'] - df_variables['p_lbound'])
# df_variables['intercept'] = df_variables['l_lsale'] - df_variables['slope'] * df_variables['p_lbound']
df_variables

,型号,价格下界,价格上界,价格下界时销量,价格上界时销量
0,115900BLK,34,35,127,60
1,115900BLU,34,40,24,20
2,115900PUR,34,45,7,4
3,115900BBU,34,41,4,2
4,115900PNK,34,42,4,2
5,115900WHT,34,43,2,1
6,115900PBU,34,44,7,4
7,115900AGN,34,45,11,1
8,115900LPR,34,46,4,2
9,115902BLK,34,47,18,10


In [29]:
dft = df_params.merge(df_variables, on='型号')

dft['型号'] = dft['型号'].str[-4:]
dft.to_excel('data/demo.xlsx',index=False)

In [35]:
df_params = dft

In [37]:
import pyomo.environ as pyo

df_params['slope'] = (df_params['价格上界时销量'] - df_params['价格下界时销量']) / (df_params['价格上界'] - df_params['价格下界'])
df_params['intercept'] = df_params['价格下界时销量'] - df_params['slope'] * df_params['价格下界']

# 创建模型
model = pyo.ConcreteModel()

def price_bounds_rule(model, i):
    return df_params.loc[i, '价格下界'], df_params.loc[i, '价格上界']

# 定义变量
model.prices = pyo.Var(range(n_product), domain=pyo.NonNegativeReals, bounds=price_bounds_rule)

# 定义目标函数
def total_profit_rule(model):
    a = df_params['slope']
    b = df_params['intercept']

    # FBA佣金（本地国)
    fba_commission = [model.prices[i] * 0.15 for i in range(n_product)]
    # 人工6%
    manual_fee = [model.prices[i] * 0.06 for i in range(n_product)]
    # 仓储成本2%
    storage_fee = [model.prices[i] * 0.02 for i in range(n_product)]
    # 实得收入
    product_income = [model.prices[i] * (1 - df_params['折损（退货率）'][i]) - df_params['最新亚马逊配送费（美金）'][i] -
                        fba_commission[i] - df_params['VAT20%\n（本地国)'][i] - df_params['促销折扣费用'][i] -
                        df_params['PPC推广费用'][i] - manual_fee[i] - storage_fee[i] for i in range(n_product)]
    # 毛利润 rmb
    product_profit_rmb = [product_income[i] * df_params['汇率'][i] - df_params['采购成本'][i] -
                            df_params['运费及税金￥'][i] for i in range(n_product)]

    profit = sum(product_profit_rmb[i] * (a[i] * model.prices[i] + b[i]) for i in range(n_product))
    return profit

model.total_profit = pyo.Objective(rule=total_profit_rule, sense=pyo.maximize)

# 求解模型
solver = pyo.SolverFactory('ipopt')
result = solver.solve(model)

# 输出优化结果
optimized_prices = [pyo.value(model.prices[i]) for i in range(n_product)]
max_profit = pyo.value(model.total_profit)

print("优化后的价格:", [f"{i:03f}" for i in optimized_prices])
print("最大利润:", max_profit)

优化后的价格: ['34.000000', '40.000000', '43.667852', '37.057234', '37.677386', '39.505409', '42.657155', '36.337408', '42.191608', '45.632904', '44.701344', '46.659585', '41.105377', '38.702561', '36.512152', '42.296319', '39.444278', '52.144553']
最大利润: 8125.709730731421


In [19]:
model.pprint()

1 Var Declarations
    prices : Size=18, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
          0 :    34 :  33.99999966002506 :    35 : False : False : NonNegativeReals
          1 :    34 :  40.00000039961672 :    40 : False : False : NonNegativeReals
          2 :    34 : 43.667852321104554 :    45 : False : False : NonNegativeReals
          3 :    34 :  37.05723393670432 :    41 : False : False : NonNegativeReals
          4 :    34 :  37.67738641675764 :    42 : False : False : NonNegativeReals
          5 :    34 :  39.50540948626991 :    43 : False : False : NonNegativeReals
          6 :    34 :  42.65715493504316 :    44 : False : False : NonNegativeReals
          7 :    34 :  36.33740781725555 :    45 : False : False : NonNegativeReals
          8 :    34 :  42.19160819104093 :    46 : False : False : NonNegativeReals
          9 :    34 :  45.63290389420885 :    47 : False : F

In [38]:
def print_expr(expr = product_income[0]):
    print("目标函数表达式:", expr)
    print("常数项 (constant):", expr.constant)
    print("线性项 (linear):", expr.linear_coefs, expr.linear_vars)
    # print("二次项 (quadratic):", expr.quadratic_coefs, expr.quadratic_vars1, expr.quadratic_vars2)
    # print("非线性项 (nonlinear):", expr.nonlinear_exprs)

print_expr(product_profit_rmb[0])

NameError: name 'product_income' is not defined

In [12]:
model.display()

Model unknown

  Variables:
    prices : Size=18, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :    34 :  None :    35 : False :  True : NonNegativeReals
          1 :    34 :  None :    40 : False :  True : NonNegativeReals
          2 :    34 :  None :    45 : False :  True : NonNegativeReals
          3 :    34 :  None :    41 : False :  True : NonNegativeReals
          4 :    34 :  None :    42 : False :  True : NonNegativeReals
          5 :    34 :  None :    43 : False :  True : NonNegativeReals
          6 :    34 :  None :    44 : False :  True : NonNegativeReals
          7 :    34 :  None :    45 : False :  True : NonNegativeReals
          8 :    34 :  None :    46 : False :  True : NonNegativeReals
          9 :    34 :  None :    47 : False :  True : NonNegativeReals
         10 :    34 :  None :    48 : False :  True : NonNegativeReals
         11 :    34 :  None :    49 : F

# 输出

In [47]:
opt_price = np.array(optimized_prices).round(2)

In [51]:
df_result = df_params.copy()

In [66]:
df_result['优化价格'] = opt_price
df_result['预期销量'] = (df_result['slope']  * df_result['优化价格'] + df_result['intercept']).round(0).astype(int)
df_result['FBA佣金（本地国)'] = df_result['优化价格']*0.15
df_result['人工6%'] = df_result['优化价格']*0.06
df_result['仓储成本2%'] = df_result['优化价格']*0.02
df_result['得到人民币'] = (df_result['优化价格'] * (1 - df_result['折损（退货率）']) - df_result['最新亚马逊配送费（美金）'] -  df_result['FBA佣金（本地国)'] - df_result['VAT20%\n（本地国)'] - df_result['促销折扣费用'] - df_result['PPC推广费用'] - df_result['人工6%'] - df_result['仓储成本2%']).round(2)

df_result['毛利润 rmb'] = (df_result['得到人民币'] * df_result['汇率'] - df_result['采购成本'] - df_result['运费及税金￥']).round(2)
df_result.drop(columns=['slope','intercept'])

,型号,采购成本,运费及税金￥,最新亚马逊配送费（美金）,VAT20%\n（本地国),折损（退货率）,促销折扣费用,PPC推广费用,汇率,价格下界,价格上界,价格下界时销量,价格上界时销量,优化价格,预期销量,FBA佣金（本地国),人工6%,仓储成本2%,得到人民币,毛利润 rmb
0,0BLK,65,10.00,7.09,0,0.1049,0,2,7.1,34,35,127,60,34.00,127,5.1000,2.0400,0.6800,13.52,20.99
1,0BLU,65,10.00,7.09,0,0.1213,0,0,7.1,34,40,24,20,40.00,20,6.0000,2.4000,0.8000,18.86,58.91
2,0PUR,67,10.00,7.09,0,0.1218,0,0,7.1,34,45,7,4,43.67,4,6.5505,2.6202,0.8734,21.22,73.66
3,0BBU,65,10.00,7.09,0,0.0940,0,0,7.1,34,41,4,2,37.06,3,5.5590,2.2236,0.7412,17.96,52.52
4,0PNK,63,10.00,7.17,0,0.0817,0,0,7.1,34,42,4,2,37.68,3,5.6520,2.2608,0.7536,18.77,60.27
5,0WHT,63,10.00,7.17,0,0.1239,0,0,7.1,34,43,2,1,39.51,1,5.9265,2.3706,0.7902,18.36,57.36
6,0PBU,69,10.00,7.17,0,0.1161,0,0,7.1,34,44,7,4,42.66,4,6.3990,2.5596,0.8532,20.73,68.18
7,0AGN,69,10.00,7.17,0,0.0815,0,0,7.1,34,45,11,1,36.34,9,5.4510,2.1804,0.7268,17.85,47.74
8,0LPR,69,10.00,7.17,0,0.0765,0,0,7.1,34,46,4,2,42.19,3,6.3285,2.5314,0.8438,22.09,77.84
9,2BLK,65,10.00,6.77,0,0.1513,0,0,7.1,34,47,18,10,45.63,11,6.8445,2.7378,0.9126,21.46,77.37


In [26]:
dt =     {
        "model": "0BLU",
        "purchase_cost": 65,
        "shipping_and_tax": 10.0,
        "latest_amazon_delivery_fee_usd": 7.09,
        "vat_local": 0,
        "damage_rate": 0.1213,
        "promotion_discount": 0,
        "ppc_cost": 0,
        "exchange_rate": 7.1,
        "price_lower_bound": 34,
        "price_upper_bound": 40,
        "sales_at_lower_bound": 24,
        "sales_at_upper_bound": 20
    }
dt.keys()

dict_keys(['model', 'purchase_cost', 'shipping_and_tax', 'latest_amazon_delivery_fee_usd', 'vat_local', 'damage_rate', 'promotion_discount', 'ppc_cost', 'exchange_rate', 'price_lower_bound', 'price_upper_bound', 'sales_at_lower_bound', 'sales_at_upper_bound'])

In [27]:
import pandas as pd 

df = pd.read_excel('data/demo.xlsx')
df.columns = ['model', 'purchase_cost', 'shipping_and_tax', 'latest_amazon_delivery_fee_usd', 'vat_local', 'damage_rate', 'promotion_discount', 'ppc_cost', 'exchange_rate', 'price_lower_bound', 'price_upper_bound', 'sales_at_lower_bound', 'sales_at_upper_bound']
data = df.to_json('data/demo.json', orient='records')

In [3]:
!curl -X POST "http://127.0.0.1:52565/optimize" -H "Content-Type: application/json" -d @data/demo.json

{"optimized_price":[34.0,40.0,43.67,37.06,37.68,39.51,42.66,36.34,42.19,45.63,44.7,46.66,41.11,38.7,36.51,42.3,39.44,52.14],"expected_sales":[127,20,4,3,3,1,4,9,3,11,5,2,8,6,2,2,2,1],"fba_commission_local":[5.1,6.0,6.5505,5.559,5.652,5.9265,6.398999999999999,5.4510000000000005,6.328499999999999,6.8445,6.705,6.999,6.1665,5.805000000000001,5.4765,6.345,5.9159999999999995,7.821],"labor_cost_6_percent":[2.04,2.4,2.6202,2.2236000000000002,2.2607999999999997,2.3705999999999996,2.5595999999999997,2.1804,2.5313999999999997,2.7378,2.682,2.7996,2.4665999999999997,2.322,2.1906,2.538,2.3663999999999996,3.1284],"storage_cost_2_percent":[0.68,0.8,0.8734000000000001,0.7412000000000001,0.7536,0.7902,0.8532,0.7268000000000001,0.8438,0.9126000000000001,0.8940000000000001,0.9331999999999999,0.8222,0.774,0.7302,0.846,0.7888,1.0428],"net_income_rmb":[13.52,18.86,21.22,17.96,18.77,18.36,20.73,17.85,22.09,21.46,21.55,21.04,19.8,19.36,18.02,17.39,17.8,25.43],"gross_profit_rmb":[20.99,58.91,73.66,52.52,60.27,5

## Deal

In [8]:
from pyomo.environ import *

# 创建模型
model = ConcreteModel()

# 定义变量范围，假设每个变量可以取值范围为1到10
value_range = list(range(1, 11))

# 定义变量
model.x = Var(range(7), domain=Integers, bounds=(1, 10))

# 定义布尔变量以表示每个可能的取值是否被使用
model.used = Var(value_range, domain=Binary)

# 添加约束：如果某个值被使用，则相关布尔变量为1
def usage_rule(model, v):
    return model.used[v] >= sum(model.x[i] == v for i in range(7))
model.usage = Constraint(value_range, rule=usage_rule)

# 添加约束：总共使用的不同值数量等于3
model.unique_value_count = Constraint(expr=sum(model.used[v] for v in value_range) == 3)

# 设置一个虚拟的目标函数（可以根据具体问题调整）
model.obj = Objective(expr=sum(model.x[i] for i in range(7)))

# 解决模型
solver = SolverFactory('glpk')  # 可以使用其他可用的求解器
results = solver.solve(model)

# 输出结果
print("Solution:")
for i in range(7):
    print(f"x{i+1} = {value(model.x[i])}")
print("Used values:")
for v in value_range:
    if value(model.used[v]) > 0:
        print(f"Value {v} is used")


ERROR: Rule failed when generating expression for Constraint usage with index
1: TypeError: unsupported operand type(s) for +: 'int' and
'EqualityExpression'
ERROR: Constructing component 'usage' from data=None failed:
        TypeError: unsupported operand type(s) for +: 'int' and
        'EqualityExpression'


TypeError: unsupported operand type(s) for +: 'int' and 'EqualityExpression'